In [4]:
import pandas as pd


In [6]:
df = pd.read_csv('../data/sentiment.csv').drop(columns=['Unnamed: 0'])

df.head()

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety
